Imports

In [88]:
import pandas as pd
from pathlib import Path
import math

Read Data

In [89]:
payslips = pd.DataFrame([], columns = ["CHQDATE", "PAYNO", "TSNO", "FORENAME", "SURNAME", "COMPNAME", "MANAGEMENT FEE", "PAYDESC", "TOTHRS", "TOTNET", "TOTPAY", "TYPE", "NI_NO", "Comp Fee Key"])

fee_codes = pd.read_csv(Path().home() / "OneDrive - advance.online/Documents/data/fee codes.csv")

for file in Path().absolute().glob("*"):
    if file.name.__contains__("Payslips") and file.suffix == ".csv":
        print(f"reading file {file.name}")
        payslips = pd.concat([payslips, pd.read_csv(file, usecols=payslips.columns, encoding="latin").dropna(subset=["MANAGEMENT FEE"])])

payslips = payslips.merge(fee_codes, left_on="Comp Fee Key", right_on="Fee Type", how="left").drop(columns="Fee Type")

reading file Payslips Historical 21 axm.csv
reading file Payslips Historical 21 io.csv
reading file Payslips Historical 21 paye.csv
reading file Payslips Historical 22 axm.csv
reading file Payslips Historical 22 io.csv
reading file Payslips Historical 22 paye.csv


In [90]:
fee_codes[fee_codes["Value"] == "formula"]

,Fee Type,Value
26,RSS1,formula
27,SCANT,formula
28,TENED,formula
30,EDU,formula
31,SCAT,formula
32,SCNPT,formula
33,TEN,formula
36,RSS,formula


In [91]:
for i, row in payslips.iterrows():
    if pd.isnull(row["Value"]):
        payslips.at[i, "Value"] = float( row["Comp Fee Key"].replace("£", "").replace(",", "") )
    elif row["Value"] == "formula":
        match row["Comp Fee Key"]:
            case "RSS1":   
                if row["TOTHRS"] <= 12:
                    payslips.at[i, "Value"] = 6.9
                elif row["TOTHRS"] <= 16:
                    payslips.at[i, "Value"] = 11.7
                elif row["TOTHRS"] <= 20:
                    payslips.at[i, "Value"] = 11.95
                elif row["TOTHRS"] <= 24:
                    payslips.at[i, "Value"] = 15.45
                elif row["TOTHRS"] <= 32:
                    payslips.at[i, "Value"] = 18.45
                elif row["TOTHRS"] <= 40:
                    payslips.at[i, "Value"] = 20.95
                elif row["TOTHRS"] <= 48:
                    payslips.at[i, "Value"] = 23.45
                elif row["TOTHRS"] <= 55:
                    payslips.at[i, "Value"] = 24.45
                else:
                    payslips.at[i, "Value"] = 26.95
                    
            case "RSS":
                if row["TOTHRS"] <= 12:
                    payslips.at[i, "Value"] = 4.95
                elif row["TOTHRS"] <= 16:
                    payslips.at[i, "Value"] = 9.75
                elif row["TOTHRS"] <= 20:
                    payslips.at[i, "Value"] = 10
                elif row["TOTHRS"] <= 24:
                    payslips.at[i, "Value"] = 13.5
                elif row["TOTHRS"] <= 32:
                    payslips.at[i, "Value"] = 16.5
                elif row["TOTHRS"] <= 40:
                    payslips.at[i, "Value"] = 19
                elif row["TOTHRS"] <= 48:
                    payslips.at[i, "Value"] = 21.5
                elif row["TOTHRS"] <= 55:
                    payslips.at[i, "Value"] = 22.5
                else:
                    payslips.at[i, "Value"] = 25
                
            case "TENED" | "EDU" | "TEN":
                if row["TOTHRS"] <= 6:
                    payslips.at[i, "Value"] = 2.88
                elif row["TOTHRS"] <= 12:
                    payslips.at[i, "Value"] = 5.76
                elif row["TOTHRS"] <= 18:
                    payslips.at[i, "Value"] = 8.64
                elif row["TOTHRS"] <= 24:
                    payslips.at[i, "Value"] = 11.52
                else:
                    payslips.at[i, "Value"] = 12
            
            case "SCANT" | "SCNPT":
                payslips.at[i, "Value"] = 14 if row["TOTHRS"] <= 24 else 26.90
                
            case "SCAT":
                payslips.at[i, "Value"] = 14 if row["TOTHRS"] <= 24 else 21.90
                
            case default:
                print(rf"Unhandled Fee Code {row['Comp Fee Key']}")
    
    if row["MANAGEMENT FEE"] > float(payslips.at[i, "Value"]) > 0:
        
        count = math.floor(row["MANAGEMENT FEE"] / float(payslips.at[i, "Value"])) if row["MANAGEMENT FEE"] >= 2*float(payslips.at[i, "Value"]) else 1
        
        payslips.at[i, "Rewards"] = row["MANAGEMENT FEE"] - count*float(payslips.at[i, "Value"])

    else:
        payslips.at[i, "Rewards"] = 0
            
        

In [92]:
payslips["Margin Count"] = (payslips["MANAGEMENT FEE"] - payslips["Rewards"]) / payslips["Value"].astype(float)

In [93]:
print(sum(payslips["Rewards"]))

146864.4900000765


In [94]:
payslips.to_csv("Rewards Margin.csv", index=False)